In [4]:
import sys
import time
import json

sys.path.append('../core/python')

from commons import inc_inference, show_heatmap
from imagenet_classes import class_names
from vgg16 import VGG16

In [5]:
image_file_path = "../images/input/imagenet/dog.jpg"

In [ ]:
'''
start the experimeng with vgg16
iterate all time-possibilities of vgg16
store all the patch,stride,height,width possibilities into .json 
as the backend for later estimate time cost
of running Krypton
'''
patch_dict = [1,2,4,8,16]
stride_dict = [1,2,4,8,16]

data = {}
data['vgg16'] = []

for pt in patch_dict:
    print('pt: ' + str(pt))
    for st in stride_dict:
        print('st: ' + str(st))
        for x in range(224):
            for y in range(224):
                begin_time = time.time()
                _,_,_ = inc_inference(VGG16, image_file_path, patch_size=pt, stride=st, beta=1.0, x0=0, y0=0, image_size=224, x_size=x,
                                      y_size=y, version='v1', gpu=True, c=0.0)
                end_time = time.time()
                time_elapsed = end_time - begin_time
                data['vgg16'].append({
                    'patch': pt,
                    'stride': st,
                    'width': x,
                    'height': y,
                    'time': time_elapsed
                })
        
        
with open('time-estimation.txt', 'w') as outfile:
    json.dump(data, outfile)

pt: 1
st: 1
